In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [ ]:
from robust_standard_errors import Robust_standard_errors
rse = Robust_standard_errors()

In [ ]:
spark = SparkSession.builder.appName("robust_standard_errors_test").getOrCreate()
data = spark.read.option("header",True).csv("sample_data/california_housing_train.csv")
data.show(truncate=False)

In [ ]:
assembler = VectorAssembler().setInputCols(['housing_median_age']).setOutputCol('features')
data = assembler.transform(data)
data.show(truncate=False)

In [ ]:
lr = LinearRegression()
lrModel = lr.fit(data)
lrModel.setPredictionCol("newPrediction")

## Umsetzen von Sandwich

#### Heteroskedastizität

In [ ]:
values = rse.robust_standard_errors_heteroskedasticity_white(lrModel,data,["housing_median_age"])
values.toRowMatrix().rows.collect()

In [ ]:
rse.get_model_summary(lrModel,values,["housing_median_age"])

#### Autokorrelation

In [ ]:
values = rse.robust_standard_errors_autocorrelation_newey_and_west(lrModel,data,["housing_median_age"])
values.toRowMatrix().rows.collect()

In [ ]:
rse.get_model_summary(lrModel,values,["housing_median_age"])